<a href="https://colab.research.google.com/github/vdyakushina/Scientific-Python/blob/main/HW_27_04_2021_Bio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install Bio

In [102]:
# 1
'''
Let's remeber how to use dictionaries.
Task: return a dictionary where 
    * keys are IDs of seqs from an input fasta file (prot.fasta),
    * key's values are seqs itself. 
'''

def my_own_fasta_parser(inFile):
  sequences = {}
  with open(inFile, "r") as inpf:
    for line in inpf:
      line=line.strip()
      if ">" in line:
        seq_id = line[1:]
      else:
        sequences[seq_id] = line
  return sequences

my_own_fasta_parser('prot.fasta')

FileNotFoundError: ignored

In [ ]:
# 2
'''
Another super easy task.

We have the same fasta file (prot.fasta).
Now we want to get a list with the ids of protein seqs that have 
a relative frequency higher than a given threshold for a given residue.

And don't forget to use my_own_fasta_parser function from a previous task!
'''

def my_own_residue_abundance(input_file, residue, threshold=0.2):
  seq_id = []
  sequences = my_own_fasta_parser(input_file)
  for key_id, sequence in sequences.items():
    freq=sequence.count(residue)/len(sequence)
    if freq > threshold:
      seq_id.append(key_id)
  return seq_id

result2 = my_own_residue_abundance('/content/drive/MyDrive/Colab Notebooks/Sci Pyth/prot.fasta', "W", threshold=0.01)

In [ ]:
# 3
'''
Let's practice in Bio package using.
Task:
1. read fasta file containing several DNA seqs (nucl.fasta)
2. subset seqs that have GC content > 45 and coding protein with aromaticity > 0.01
3. write a new fasta file with those protein(!) seqs
4. return the percentage of seqs that passed your filter
Hint: Bio.SeqIO, Bio.SeqRecord, Bio.SeqUtils
'''

import Bio.SeqIO
import Bio.SeqRecord
import Bio.SeqUtils
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Seq import Seq

def my_own_filtering(input_file, output_file, filt_gc=45, filt_arom=0.125):
  sequences = {}
  c = 0
  with open(input_file, "r") as content:
    for record in Bio.SeqIO.parse(content, "fasta"):
      c+=1
      gcn=Bio.SeqUtils.GC(record.seq)
      aroma=ProteinAnalysis(str(record.seq.translate())).aromaticity()
      if gcn >= filt_gc and aroma >= filt_arom:
        sequences[record.id] = record.seq
  records = []
  for seq_id, seq in sequences.items():
    records.append(Bio.SeqRecord.SeqRecord(Seq(str(seq.translate())), id=seq_id, description=""))
  file = open(output_file,"w")
  Bio.SeqIO.write(records, file, "fasta")
  file.close()
  print(f'percentage of seqs that passed filter: {(len(records)/c)*100}')

my_own_filtering('nucl.fasta', 'nucl_out.fasta', filt_gc=45, filt_arom=0.125)

percentage of seqs that passed filter: 14.285714285714285


/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [37]:
# 4
"""
Continue practicing in Bio package using:
Task:
complete the following code that should be able to return 
the best alignment of two amino acid seqs (pairwise2.align.globalds)
based on BLOSUM62 matrix from Bio.SubsMat.MatrixInfo.
http://rosalind.info/glossary/blosum62/
"""

from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist

def balign(first_seq, second_seq):
    matrix = matlist.blosum62
    alns = pairwise2.align.globaldx(first_seq, second_seq, matrix)
    top_aln = alns[0]
    print(f'{top_aln.seqA}, {top_aln.seqB}, {top_aln.score}, {top_aln.start}, {top_aln.end}')

balign("ACCGT", "ACG")

In [85]:
# 5
""" You have some DNA sequence: AGTACTAGAGCATTCTATGGAG.
Find out which peptides could be created from it and sort them by their length.
Use as much Biopython modules as possible.
"""

from Bio.Seq import Seq
from Bio.Data import CodonTable

my_seq = Seq("AGTACTAGAGCATTCTATGGAG")
peptides=[]
peptides.append(str(my_seq.translate(to_stop = True)))
peptides.append(str(my_seq.reverse_complement().translate(to_stop = True)))
for i in CodonTable.generic_by_id[5].start_codons:
  if my_seq.transcribe().find(i)!=-1:
    start=my_seq.transcribe().find(i)
    peptides.append(str(my_seq.transcribe()[start:].translate(to_stop = True)))

peptides=sorted(peptides, key=len, reverse=True)
for i in peptides:
  print(i)

STRAFYG
LHRML
ILW
ME


/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [101]:
# 6
""" TASK: Try to create one-line function (without (!!!) using Bio package) 
that returns reverse complementary to a given sequence. 
Hint: using dictionaty & list comprehensions might be helpful.
"""

def rev_compl_one_line(seq):
    return "".join({"A":"T", "T":"A", "C":"G", "G":"C"}[i] for i in my_seq[::-1])

rev_compl_one_line('AGTACTAGAGCATTCTATGGAG')

'CTCCATAGAATGCTCTAGTACT'